In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.impute import SimpleImputer

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from sklearn.svm import SVR

from sklearn.pipeline import Pipeline, FeatureUnion

from sklearn.metrics import mean_squared_error

import functions as f
import classes as c

In [2]:
train_df = pd.read_csv("datasets/train.csv", na_values=np.NaN).set_index("Id")
test_df = pd.read_csv("datasets/test.csv", na_values=np.NaN).set_index("Id")
c.ConflictingDataCleaner(inplace=True).fit_transform(train_df)
corr_matrix = train_df.corr()
# train_df[pd.isnull(train_df)] = np.NaN
X = train_df.drop("SalePrice", axis=1).copy()
y = train_df["SalePrice"].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [3]:
train_incomplete_col = f.columns_with_missing_values(train_df)
test_incomplete_col = f.columns_with_missing_values(test_df)
incomplete_col_diff = np.setdiff1d(train_incomplete_col, test_incomplete_col), np.setdiff1d(test_incomplete_col, train_incomplete_col)

drop_col = []

nominal_col = ["MSSubClass", "MSZoning", "LandContour", "LotConfig",
               "Neighborhood", "Condition1", "Condition2", "BldgType", "HouseStyle",
               "RoofStyle", "RoofMatl", "Exterior1st", "Exterior2nd", "MasVnrType",
               "Foundation", "Heating", "Electrical", "GarageType", "Fence", "MiscFeature", 
              "SaleType", "SaleCondition"]
binary_col = ["Street", "CentralAir"]
ordinal_col = ["Alley", "LotShape", "Utilities", "LandSlope", "OverallQual", "OverallCond",
               "ExterQual", "ExterCond", "BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1",
               "BsmtFinType2", "HeatingQC", "KitchenQual", "Functional", "FireplaceQu",
               "GarageFinish", "GarageQual", "GarageCond", "PavedDrive", "PoolQC"]
count_col = ["BsmtFullBath", "BsmtHalfBath", "FullBath", "HalfBath", "BedroomAbvGr", "KitchenAbvGr",
                "TotRmsAbvGrd", "Fireplaces", "GarageCars"]
interval_col = ["LotFrontage", "LotArea", "MasVnrArea", "BsmtFinSF1", "BsmtFinSF2", "BsmtUnfSF",
                "TotalBsmtSF", "1stFlrSF", "2ndFlrSF", "LowQualFinSF", "GrLivArea",
                "GarageArea", "WoodDeckSF", "OpenPorchSF", "EnclosedPorch", "3SsnPorch",
                "ScreenPorch", "PoolArea", "MiscVal"]
time_col = ["YearBuilt", "YearRemodAdd", "GarageYrBlt", "MoSold", "YrSold"]

cat_num_col = ["MSSubClass"]
miss_vall_col = ["LotFrontage", "MasVnrArea", "MasVnrType", "Electrical", "GarageYrBlt"]
na_cat_col = ["Alley", "BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1",
                  "BsmtFinType2", "FireplaceQu", "GarageType", "GarageFinish", "GarageQual",
                  "GarageCond", "PoolQC", "Fence", "MiscFeature"]

nominal_col_cats = f.unique_values(train_df[nominal_col])
binary_col_cats = f.unique_values(train_df[binary_col])
ordinal_col_cats = f.unique_values(train_df[ordinal_col])

# one_hot_col = nominal_col + 
# label_enc_col = binary_col + ordinal_col

In [4]:
count_interval_time_pip = Pipeline([
                ("selector", c.DataFrameSelector(columns=interval_col)),
                ("standard", StandardScaler())
            ])
nominal_pip = Pipeline([
                ("selector", c.DataFrameSelector(columns=nominal_col)),
                ("one_hot_encoder", OneHotEncoder(handle_unknown="ignore", categories=nominal_col_cats, sparse=False))
            ])
binary_ordinal_pip = Pipeline([
                ("selector", c.DataFrameSelector(columns=binary_col+ordinal_col)),
                ("one_hot_encoder", OneHotEncoder(handle_unknown="ignore", categories=binary_col_cats+ordinal_col_cats, sparse=False))
            ])
missing_vallue_pip = Pipeline([
                ("selector", c.DataFrameSelector(columns=X_train.columns, inplace=False)),
                ("mas_vnr_imputer", c.ConstantImputer(columns=["MasVnrArea", "MasVnrType"], string_fill_val="None", inplace=True)),
                ("electrical_imputer", c.DataFrameImputer(strategy="most_frequent", columns=["Electrical"], inplace=True)),
                ("garage_yr_imputer", c.GarageYrImputer(inplace=True)),
                ("lot_frontage_imputer", c.LotFrontageImputer(inplace=True)),
                ("all_constant_imputer", c.ConstantImputer(columns=X_train.columns, inplace=True))
            ])


In [5]:
full_pip = Pipeline([
    ("missing_vallue", missing_vallue_pip),
    ("feature_union", FeatureUnion([
        ("count_interval_time", count_interval_time_pip),
        ("nominal", nominal_pip),
        ("binary_ordinal_pip", binary_ordinal_pip)]))
])

In [7]:
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
clf = GridSearchCV(SVR(), tuned_parameters, cv=5, scoring="neg_mean_squared_error", verbose=1)

In [8]:
X_train_tr = full_pip.fit_transform(X_train)
X_test_tr = full_pip.transform(X_test)

clf.fit(X_train_tr, y_train)
print("Best parameters set found on development set:")
print(clf.best_params_)

C:\Users\Paul\AppData\Local\conda\conda\envs\handson-ml\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Paul\AppData\Local\conda\conda\envs\handson-ml\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\Paul\AppData\Local\conda\conda\envs\handson-ml\lib\site-packages\sklearn\pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 28 candidates, totalling 140 fits


[Parallel(n_jobs=1)]: Done 140 out of 140 | elapsed:  1.1min finished
C:\Users\Paul\AppData\Local\conda\conda\envs\handson-ml\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Best parameters set found on development set:
{'C': 1000, 'kernel': 'linear'}


In [10]:
model = clf.best_estimator_
y_train_pr = model.predict(X_train_tr)
y_test_pr = model.predict(X_test_tr)
mse_train = mean_squared_error(y_train, y_train_pr)
mse_test = mean_squared_error(y_test, y_test_pr)
np.sqrt(mse_train), np.sqrt(mse_test)

(27784.184326718725, 37151.07113029916)

In [ ]:
mse_train = mean_squared_error(y_train, y_train_pr)

In [ ]:
mse_test = mean_squared_error(y_test, y_test_pr)

In [ ]:
np.sqrt(mse_train), np.sqrt(mse_test)

In [ ]:
y.hist()

In [ ]:
df = pd.DataFrame({"n": [1, 2, 3, 4, 2, 3], "s": ["a", "b", "c", "d", "b", "c"]})

In [ ]:
df.drop(df.loc[df["n"] == 2].index, inplace=True)

In [ ]:
df

In [ ]:
df_ohe = c.DataFrameOneHotEncoder(categories=f.unique_values(train_df[["MSSubClass", "MSZoning", "LandContour"]]), columns=["MSSubClass", "MSZoning", "LandContour"])